- trf
    - ca37: distance=passthrough
    - fa15: distance=pwr
- imp
    - 2474: tune=False
    - 20cf: tune=True
- clf
    - 2c44: ensemble=True, time=20
    - 25f2: ensemble=False, time=20

In [1]:
from LiveAMP import *
# Flags().run()
# print({k:len(v) for k,v in param_dcts.items()})
self = run_amp(131)
# self = get_stack(137, '_2024_05_07')

2
creating Y/n/_allcrse/202108/fa15/1bfb/ada2.pkl: 3


In [1]:
delattr

<function delattr(obj, name, /)>

In [ ]:
self.mlt.query("crse_code.isin(['chem1407','kine2315','kine2319','kine3319','fina1360'])").sort_values(['styp_code','crse_code','pred_code']).reset_index()

In [ ]:
self.y

In [ ]:
df = pd.DataFrame()
df['a'] = [1,2,3]
len(df.query("a==1"))

In [ ]:
read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/137/X_proc/n/fa15/1bfb.pkl').keys()

In [ ]:
self.mlt.query("crse_code=='chem1407'").prep()